# Optimization 1
## Benjamin Parsons

## Question A

Formulate a schedule that __minimizes the total under-allocation of each department to operating rooms.__ For  example, General Surgery currently  receives 48.4% of the total operating room time, and should therefore receive 48.4% of the total operating room time in the new schedule as well. In particular, the CEO has specified that under-allocation is to be avoided, meaning a penalty should be incurred if a department is allocated __less__ than its target %, but there should be no penalty if a department is allocated __more__ than its target % of operating room time.  

Your model should produce a new schedule that minimizes the total under-allocation (on  apercentage basis). The reason for representing allocation on a percentage basis is that usingunits of time is not equitable: For example, a loss of 1 hour per week is much more disruptiveto Oral Surgery (currently 10 hrs/week) than it is for General Surgery (92 hrs/week).

#### Decision Variables:  
$x_{i_{department} k_{day} j_{room} }$: binary variable indicating if a department is assigned to use a room at a particular time.  
$add\_week\_hrs_{ij}$: sum of a departments assigned hours in a particular room for the week.  
$assigned\_hrs_i$: sum of a departments assgined hours.  
$u_i$: sum of a departments underassigned hours.  

#### Constraints:  
$total\_hrs = 213.5$  
$share_i = [.484, .042, .253, .074, .053, .095]$  
$target\_hrs_i = share_i * total\_hrs$  
$avail\_hrs_kj = \begin{bmatrix} 9 & 9 & 9 & 9 & 7.5 \\ 9 & 9 & 9 & 9 & 7.5 \\ 9 & 9 & 9 & 9 & 7.5 \\ 9 & 9 & 9 & 9 & 7.5 \\ 9 & 8 & 8 & 8 & 6.5 \end{bmatrix}$  
$\sum_{i=1}^6x_{ikj} = 1$ : Only one department per room per day  
$add\_week\_hrs_{ij} = \sum_{k=1}^5avail\_hrs_{kj} * x_{ikj}$ : sum of a departments assigned hours in a particular room for the week   

$assigned\_hrs_i = \sum_{j=1}^5add\_week\_hrs_{ij}$ : sum of a departments assgined hours   
$u_i \geq target\_hrs_i - assigned\_hrs_i$  
$u_i \geq 0$ : forces the underassigned hours to be equal to its underassignemnt if positive, or 0 if overassigned

#### Objective Function  
$min \sum_{i=1}^6 \frac{u_i}{target\_hrs_i}$  

In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np

# Define model and parameters. 
mod = Model()

Using license file /Users/benjaminparsons/gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-07


In [2]:
# Variable Identifiers
department = range(6)
room = range(5)
weekday = range(5)

# Parameters
total_hrs = 213.5
share = np.array([.484, .042, .253, .074, .053, .095])
target_hrs = share * total_hrs

avail_hrs = np.array([[9, 9, 9, 9, 7.5],
                      [9, 9, 9, 9, 7.5],
                      [9, 9, 9, 9, 7.5],
                      [9, 9, 9, 9, 7.5],
                      [9, 8, 8, 8, 6.5]])

In [3]:
# Decision variables
x = mod.addVars(len(department), len(weekday), len(room), vtype = GRB.BINARY)
add_week_hrs = mod.addVars(len(department), len(room))
assigned_hrs = mod.addVars(len(department))
u = mod.addVars(len(department))

In [4]:
# Constraint restriciting one department per room per weekday
one_room_week_con = {}
for j in room:
    one_room_week_con[j] = {}
    for k in weekday:
       one_room_week_con[j][k] = mod.addConstr(sum([x[i,k,j] for i in department]) == 1)

In [5]:
# Constraint assigning the sum of a departments hrs to a assigned_hrs variable
assigned_week_hrs = {}
assigned_room_hrs = {}

for i in department:
    for j in room:
        assigned_week_hrs[j] = mod.addConstr(sum([x[i,k,j] * avail_hrs[k,j] for k in weekday]) == add_week_hrs[i,j])
    assigned_room_hrs[i] = mod.addConstr(sum([add_week_hrs[i,j] for j in room]) == assigned_hrs[i])

In [6]:
# Constraint defining the minimization problem to only focus on minimizing under allocation
und_hrs_con = {}
for i in department:
    und_hrs_con[i] = mod.addConstr(target_hrs[i] - assigned_hrs[i] <= u[i])
    
und_hrs_con_none = {}
for i in department:
    und_hrs_con_none[i] = mod.addConstr(0 <= u[i])

In [7]:
# Set objective function
mod.setObjective(sum(u[i]/target_hrs[i] for i in department), GRB.MINIMIZE)

In [8]:
# Update and solve
mod.update()
mod.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 73 rows, 192 columns and 384 nonzeros
Model fingerprint: 0x8667cd66
Variable types: 42 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 42 rows and 36 columns
Presolve time: 0.00s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 4 continuous, 152 integer (150 binary)
Found heuristic solution: objective 0.6795267
Found heuristic solution: objective 0.1580699

Root relaxation: objective 2.066116e-03, 54 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00207    0    9    0.15807    0.00207  98.7%     -    0s
H    0     0

In [9]:
mod.objval

0.05190597648318124

In [10]:
print(target_hrs)
print([assigned_hrs[i].x for i in department])

[103.334    8.967   54.0155  15.799   11.3155  20.2825]
[98.0, 9.0, 54.0, 16.0, 15.0, 21.5]


In [11]:
# Produce Schedule
department_dic = {0: "General Surgery",
                1: "Emergency",
                2: "Neurosurgery",
                3: "Opthamology",
                4: "Oral Surgery",
                5: "Otolaryngology"}

In [12]:
import pandas as pd

df = pd.DataFrame(
	[[department_dic[i] for j in room for i in department if x[i,0,j].x == 1],
    [department_dic[i] for j in room for i in department if x[i,1,j].x == 1],
    [department_dic[i] for j in room for i in department if x[i,2,j].x == 1],
    [department_dic[i] for j in room for i in department if x[i,3,j].x == 1],
    [department_dic[i] for j in room for i in department if x[i,4,j].x == 1]],
	columns=["Main-1", "Main-2", "Main-3", "Main-4", "Main-5"], 
    index =["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"])
df

,Main-1,Main-2,Main-3,Main-4,Main-5
Monday,Emergency,Neurosurgery,General Surgery,General Surgery,Otolaryngology
Tuesday,General Surgery,General Surgery,General Surgery,Neurosurgery,Oral Surgery
Wednesday,Neurosurgery,General Surgery,General Surgery,Neurosurgery,Oral Surgery
Thursday,General Surgery,General Surgery,General Surgery,Neurosurgery,Otolaryngology
Friday,Neurosurgery,General Surgery,Opthamology,Opthamology,Otolaryngology


## Question B   
Operating rooms Main-1, Main-2 will be located on the first floor of the new hospital, Main-3 and  Main-4 will be on the second floor, and  Main-5 will be located on the third floor. To improve communication and mobility among department staff, the CEO has inquired whether it is possible to devise the schedule so that no department is split between two or more floors on the same day. For example, it is acceptable if a department is exclusively assigned to Main-1 onMonday and then Main-5 on Tuesday, but not acceptable if a department is assigned to Main-1 and Main-5 on the same day. Incorporate constraints into your base model from part a) to ensure that no department is allocated rooms on two different floors on the same day.

#### Additional Constraints:  
$\sum_{i=1}^6\sum_{k=1}^5x_{i1k} + x_{i3k} + x_{i5k}\leq 1$  
$\sum_{i=1}^6\sum_{k=1}^5x_{i2k} + x_{i3k} + x_{i5k}\leq 1$  
$\sum_{i=1}^6\sum_{k=1}^5x_{i1k} + x_{i4k} + x_{i5k}\leq 1$  
$\sum_{i=1}^6\sum_{k=1}^5x_{i2k} + x_{i3k} + x_{i5k}\leq 1$  

These constraints force the sum of a departments assigned binary variables to be less than or equal to 1 accross different floors


In [13]:
# Additional constraints
level_con_one = {}
for i in department:
    level_con_one[i] = {}
    for k in weekday:
        level_con_one[i][k] = mod.addConstr(x[i,k,0]+x[i,k,2]+x[i,k,4] <= 1)

level_con_two = {}
for i in department:
    level_con_two[i] = {}
    for k in weekday:
        level_con_two[i][k] = mod.addConstr(x[i,k,0]+x[i,k,3]+x[i,k,4] <= 1)

level_con_three = {}
for i in department:
    level_con_three[i] = {}
    for k in weekday:
        level_con_three[i][k] = mod.addConstr(x[i,k,1]+x[i,k,2]+x[i,k,4] <= 1)

level_con_four = {}
for i in department:
    level_con_four[i] = {}
    for k in weekday:
        level_con_four[i][k] = mod.addConstr(x[i,k,1]+x[i,k,3]+x[i,k,4] <= 1)

In [14]:
mod.update()
mod.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 193 rows, 192 columns and 744 nonzeros
Model fingerprint: 0x3829dd84
Variable types: 42 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R74 by 1.000000000

Presolve removed 37 rows and 31 columns
Presolve time: 0.00s
Presolved: 156 rows, 161 columns, 676 nonzeros
Variable types: 4 continuous, 157 integer (150 binary)
Found heuristic solution: objective 1.6709699

Root relaxation: objective 1.387152e-01, 68 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/

In [15]:
mod.objval

0.13871523409526393

In [16]:
print(target_hrs)
print([assigned_hrs[i].x for i in department])

[103.334    8.967   54.0155  15.799   11.3155  20.2825]
[89.0, 9.0, 56.5, 18.0, 18.0, 23.0]


In [17]:
floor_df = pd.DataFrame(
	[[department_dic[i] for j in room for i in department if x[i,0,j].x == 1],
    [department_dic[i] for j in room for i in department if x[i,1,j].x == 1],
    [department_dic[i] for j in room for i in department if x[i,2,j].x == 1],
    [department_dic[i] for j in room for i in department if x[i,3,j].x == 1],
    [department_dic[i] for j in room for i in department if x[i,4,j].x == 1]],
	columns=["Main-1", "Main-2", "Main-3", "Main-4", "Main-5"], 
    index =["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"])
floor_df

,Main-1,Main-2,Main-3,Main-4,Main-5
Monday,General Surgery,General Surgery,Otolaryngology,Emergency,Neurosurgery
Tuesday,Neurosurgery,Neurosurgery,General Surgery,General Surgery,Otolaryngology
Wednesday,General Surgery,General Surgery,Opthamology,Oral Surgery,Neurosurgery
Thursday,General Surgery,General Surgery,Oral Surgery,Opthamology,Neurosurgery
Friday,General Surgery,General Surgery,Neurosurgery,Neurosurgery,Otolaryngology
